## Imports

In [198]:
import pandas as pd
import requests
from urllib.request import Request, urlopen    
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from datetime import datetime



# Acquisition

### Helper functions

In [199]:
#A function that returns a dataframe of all the historic data of a given ticker. 
def get_yfinance_data(ticker):
	url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1=994016928&period2=1642982400&interval=1d&events=history&includeAdjustedClose=true'

	data = pd.read_csv(url)

	return data

### Retrieval

In [200]:
#Get crypto historic prices
df_btc_usd = get_yfinance_data('BTC-USD')
df_eth_usd = get_yfinance_data('ETH-USD')
df_sol_usd = get_yfinance_data('SOL1-USD')
df_ada_usd = get_yfinance_data('ADA-USD')
df_doge_usd = get_yfinance_data('DOGE-USD')
df_xrp_usd = get_yfinance_data('XRP-USD')


In [201]:
crypto_dfs = [df_sol_usd, df_ada_usd, df_doge_usd, df_eth_usd, df_xrp_usd]

# Preprocessing

### Helper functions

In [202]:
#Function that returns the oldest data from a list of data frames. 
def str_to_datetime(dataframes, date_column):
	for df in dataframes:
		df[date_column] = pd.to_datetime(df[date_column]).dt.date

#Function that returns the first date of the "youngest" of dataframes
def get_min_possible_date(dataframes, date_column):

	return max([min(dataframe[date_column]) for dataframe in dataframes])


#DONT USE, Function that returns the oldest date along with the name of the df that contains it
def get_oldest_df(dataframes, date_column):
	ol_date = (get_min_possible_date(dataframes, date_column), 42)
	for i, df in enumerate(dataframes):
		first_entry = min(df[date_column])
		if first_entry < ol_date[0]:
			ol_date = (first_entry, i)
	
	return ol_date[0], dataframes[ol_date[1]]

### Crypto Index

In [203]:
#Turn Date columns from str to datetime format
str_to_datetime(crypto_dfs, 'Date')

We know that bitcoin is the oldest cryptocurrency so we can calculate the index with it as a base

In [204]:
df_btc_usd = df_btc_usd[['Date', 'Close', 'Volume']].rename({'Close': 'BTC_price', 'Volume': 'BTC_vol'}, axis=1)
df_eth_usd = df_eth_usd[['Date', 'Close', 'Volume']].rename({'Close': 'ETH_price', 'Volume': 'ETH_vol'}, axis=1)
df_sol_usd = df_sol_usd[['Date', 'Close', 'Volume']].rename({'Close': 'SOL_price', 'Volume': 'SOL_vol'}, axis=1)
df_ada_usd = df_ada_usd[['Date', 'Close', 'Volume']].rename({'Close': 'ADA_price', 'Volume': 'ADA_vol'}, axis=1)
df_doge_usd = df_doge_usd[['Date', 'Close', 'Volume']].rename({'Close': 'DOGE_price', 'Volume': 'DOGE_vol'}, axis=1)
df_xrp_usd = df_xrp_usd[['Date', 'Close', 'Volume']].rename({'Close': 'XRP_price', 'Volume': 'XRP_vol'}, axis=1)


# crypto_index = df_btc_usd[['Date','Close', 'Volume']]
# crypto_index = pd.concat([crypto_index,])

In [208]:
df_eth_usd.dtypes

Date          object
ETH_price    float64
ETH_vol        int64
dtype: object

In [205]:
cr_index = df_btc_usd
cr_index = cr_index.merge(df_eth_usd, on='Date')

In [206]:
cr_index

,Date,BTC_price,BTC_vol,ETH_price,ETH_vol
